In [1]:
import pandas as pd
import pandasql
url = 'https://docs.google.com/spreadsheets/d/1k4z-q_DRc3chyf1jYqOCV4UfGGiRgC6FJ4K1e_qZUZ8/edit?usp=sharing'
from oauth2client.service_account import ServiceAccountCredentials
import gspread

scope = ['https://www.googleapis.com/auth/spreadsheets']
credentials = ServiceAccountCredentials.from_json_keyfile_name('light-field-305907-45186846c91a.json', scope)
client = gspread.authorize(credentials)
sheet = client.open_by_url(url)
worksheet = sheet.get_worksheet(0)
results = worksheet.get_all_records()

In [2]:
result_df = pd.DataFrame(results)

In [3]:
result_df.columns

Index(['ประทับเวลา', 'ชื่อทีมที่รับการประเมิน',
       'ความสมเหตุสมผล ของคำตอบที่เพื่อน comment',
       'การประเมินข้อจำกัด หรือความเสี่ยง ที่อาจทำให้ผลิตภัณฑ์ไม่สามารถใช้งานได้',
       'ความเหมาะสมของการหาข้อมูลลักษณะของรายได้',
       'ความเหมาะสมของการหาข้อมูลคู่แข่งของผลิตภัณฑ์', 'ข้อเสนอแนะเพิ่มเติม',
       'ให้ความคิดเห็นโดยทีม......'],
      dtype='object')

In [6]:
result_df.loc[result_df.ให้ความคิดเห็นโดยทีม == 'อะหรือ..อะหรือ', 'ให้ความคิดเห็นโดยทีม'] = 'อะหรือ อะหรือ'
result_df.loc[result_df.ให้ความคิดเห็นโดยทีม == 'อะหรืออะหรือ', 'ให้ความคิดเห็นโดยทีม'] = 'อะหรือ อะหรือ'
result_df.loc[result_df.ให้ความคิดเห็นโดยทีม == 'โรตี ไข่ดาว', 'ให้ความคิดเห็นโดยทีม'] = 'โรตีไข่ดาว'

In [7]:
result_df.columns.to_list()

['ประทับเวลา',
 'ชื่อทีมที่รับการประเมิน',
 'ความสมเหตุสมผล ของคำตอบที่เพื่อน comment',
 'การประเมินข้อจำกัด หรือความเสี่ยง ที่อาจทำให้ผลิตภัณฑ์ไม่สามารถใช้งานได้',
 'ความเหมาะสมของการหาข้อมูลลักษณะของรายได้',
 'ความเหมาะสมของการหาข้อมูลคู่แข่งของผลิตภัณฑ์',
 'ข้อเสนอแนะเพิ่มเติม',
 'ให้ความคิดเห็นโดยทีม......',
 'ให้ความคิดเห็นโดยทีม']

In [8]:
column_name = result_df.columns.to_list()

In [9]:
result_df.to_csv('result.csv', encoding='utf8', columns = column_name, index=False)

In [10]:
q1 = 'sum([ความสมเหตุสมผล ของคำตอบที่เพื่อน comment])/count(*) as q1'
q2 = 'sum([การประเมินข้อจำกัด หรือความเสี่ยง ที่อาจทำให้ผลิตภัณฑ์ไม่สามารถใช้งานได้])/count(*) as q2'
q3 = 'sum([ความเหมาะสมของการหาข้อมูลลักษณะของรายได้])/count(*) as q3'
q4 = 'sum([ความเหมาะสมของการหาข้อมูลคู่แข่งของผลิตภัณฑ์])/count(*) as q4'

In [11]:
sql = "select ชื่อทีมที่รับการประเมิน, %s, %s, %s, %s from result_df group by ชื่อทีมที่รับการประเมิน" % (q1, q2, q3, q4)

In [12]:
sql

'select ชื่อทีมที่รับการประเมิน, sum([ความสมเหตุสมผล ของคำตอบที่เพื่อน comment])/count(*) as q1, sum([การประเมินข้อจำกัด หรือความเสี่ยง ที่อาจทำให้ผลิตภัณฑ์ไม่สามารถใช้งานได้])/count(*) as q2, sum([ความเหมาะสมของการหาข้อมูลลักษณะของรายได้])/count(*) as q3, sum([ความเหมาะสมของการหาข้อมูลคู่แข่งของผลิตภัณฑ์])/count(*) as q4 from result_df group by ชื่อทีมที่รับการประเมิน'

In [13]:
sum_report =  pandasql.sqldf(sql)

sum_report.head(20)

,ชื่อทีมที่รับการประเมิน,q1,q2,q3,q4
0,SU Wake Up,7,7,7,7
1,We Bear Bear,6,6,7,7
2,ข้าวเหนียวมะม่วง,5,5,6,6
3,ปังปุริเย่,6,6,6,5
4,ป่งป๊ง,6,6,6,6
5,ศิลปะยืนยาวเวลานอนสั้น,5,7,7,7
6,อะหรือ อะหรือ,7,6,6,6
7,แม่หนูกกลัวโดเรม่อน,5,7,7,7
8,โรตีไข่ดาว,7,7,6,6


In [14]:
sum_report['sum'] = sum_report.sum(axis=1)

In [15]:
sum_report.head(20)

,ชื่อทีมที่รับการประเมิน,q1,q2,q3,q4,sum
0,SU Wake Up,7,7,7,7,28
1,We Bear Bear,6,6,7,7,26
2,ข้าวเหนียวมะม่วง,5,5,6,6,22
3,ปังปุริเย่,6,6,6,5,23
4,ป่งป๊ง,6,6,6,6,24
5,ศิลปะยืนยาวเวลานอนสั้น,5,7,7,7,26
6,อะหรือ อะหรือ,7,6,6,6,25
7,แม่หนูกกลัวโดเรม่อน,5,7,7,7,26
8,โรตีไข่ดาว,7,7,6,6,26
